<center>
<img src="https://drive.google.com/uc?id=1f1gGVI-rxcHjA90WEGNvvtSXF1pAxQwg" alt="Fasilkom UI" width="300"/>

CSGE603130 • Kecerdasan Artifisial dan Sains Data Dasar

Semester Ganjil 2021/2022

Fakultas Ilmu Komputer, Universitas Indonesia

##**Lab 2: *Statistics & Data Visualization***

###**Tenggat Waktu: 18 September 2021, 23.55 WIB**
</center>

####**Ketentuan:**

1. Dokumen *template* lab dengan format .ipynb dan dataset (jika dibutuhkan) telah disediakan di SCeLe
2. Jalankan kode pada dokumen .ipynb dan perhatikan dengan saksama apa yang potongan kode tersebut lakukan beserta dengan keluarannya. Jawablah **pertanyaan yang disisipkan** pada potongan program yang diberikan.
3. Dokumen Jupyter Notebook yang telah dilengkapi dengan jawaban dikumpulkan dengan format penamaan **Kelas_LabX_NPM_Nama.ipynb**. Contoh: F_Lab2_1706979341_Lulu Ilmaknun Qurotaini.ipynb
4. Kumpulkan dokumen tersebut pada submisi yang telah disediakan di SCeLe sesuai dengan kelas masing-masing sebelum **18 September 2021, 23.55 WIB**. Keterlambatan pengumpulan akan dikenakan pinalti.
5. Lab ini dirancang sebagai **tugas mandiri**. Plagiarisme tidak diperkenankan dalam bentuk apapun. Adapun kolaborasi berupa diskusi (tanpa menyalin maupun mengambil jawaban orang lain) dan literasi masih diperbolehkan dengan mencantumkan kolaborator dan sumber.

## **Pernyataan Integritas**

Wajib diisi. Tanpa pernyataan integritas submisi akan dikenakan penalti.

In [ ]:
# Isi dengan data diri Anda
NAMA = ""
KELAS = ""
NPM = ""

# Isi dengan NPM teman yang berdiskusi dengan Anda
KOLABORATOR = []

# Isi dengan sumber referensi yang Anda gunakan dalam mengerjakan
REFERENSI = []

In [ ]:
PERNYATAAN_INTEGRITAS = "Saya, %s dari kelas %s dengan NPM %s, menyatakan bahwa seluruh jawaban pada pekerjaan ini murni saya kerjakan sendiri.\n\
Saya tidak mencontek jawaban, memberikan jawaban, maupun menyalin dari sumber manapun.\n \
\n\
Jika saya melanggar pernyataan tersebut, saya siap menerima konsekuensi apapun yang diberikan.\n   \
(%s)" % (NAMA, KELAS, NPM, NAMA)

print(PERNYATAAN_INTEGRITAS)

# Exploratory Data Analysis

*Exploratory Data Analysis* (EDA) merupakan suatu proses menganalisis dataset untuk melihat karakteristik utama dari data. EDA adalah pemrosesan dasar yang dapat mempengaruhi keputusan-keputusan seorang data scientist dalam memproses data di tahap selanjutnya. Proses ini sering melibatkan dua hal, di antaranya *descriptive statistics* dan data *visualization*.

## Deskripsi Dataset Avocado

Tabel di bawah menunjukkan data pemindaian ritel mingguan 2018 untuk volume (unit) dan harga ritel Nasional. Data pemindaian ritel datang langsung dari pengecer berdasarkan penjualan eceran asli alpukat jenis Hass. Mulai tahun 2013, tabel di bawah ini mencerminkan kumpulan data ritel multi-outlet. AveragePrice di tabel mencerminkan biaya per unit (per alpukat). Kode Pencarian Produk (PLU) di tabel hanya untuk alpukat Hass. Varietas alpukat lainnya (misalnya kulit hijau) tidak termasuk dalam tabel ini.

* `Date`: the date of the observation
* `AveragePrice`: the average price of a single avocado
* `Total Volume`: total weight of avocados sold
* `4046`: total weight of avocados with PLU 4046 sold
* `4225`: total weight of avocados with PLU 4225 sold
* `4770`: total weight of avocados with PLU 4770 sold
* `Total Bags`: total weight of sold avocados put in bags (excluding those with PLUs)
* `Small Bags`: total weight of sold avocados put in small bags
* `Large Bags`: total weight of sold avocados put in large bags
* `XLarge Bags`: total weight of sold avocados put in extra large bags
* `type`: conventional or organic
* `year`: the year
* `region`: the city or region of the observation


In [ ]:
import pandas as pd
import numpy as np
import seaborn as sns
import plotly.express as px
import matplotlib.pyplot as plt
import scipy.stats as scp
import statsmodels.api as sm

In [ ]:
# Import dataset dengan index di kolom pertama
avocado_df = pd.read_csv('avocado.csv', index_col=0)

In [ ]:
total_rows, total_attributes = avocado_df.shape
print('Jumlah data:', total_rows)
print("Jumlah atribut:", total_attributes)

avocado_df.head()

In [ ]:
# Melihat kolom-kolom pada dataframe
avocado_df.info()

Terlihat sekilas tipe dan jumlah non-null dari setiap kolom. Dari informasi ini, kita mengetahui tidak ada kolom yang memiliki null untuk dataset avocado (karena semua kolom memiliki jumlah yang sama dengan jumlah data).

In [ ]:
# Menampilkan statistics untuk setiap atribut numerik 
avocado_df.describe()

In [ ]:
# Menampilkan statistics untuk setiap atribut non-numerik atau kategorik
avocado_df.describe(include=np.object)

## Descriptive Statistics

Ada banyak jenis measures yang dapat diterapkan untuk *descriptive statistics*.

Mengingat materi dari StatProb, ada
1. *Central tendency* (mean, median, mode)
2. *Dispersion* atau *variability* (variance, standard deviation)
3. Minimum, maximum
4. Range
5. Quantile
6. Percentile

**Central tendency sering dipakai untuk melihat distribusi data.**
Pada biasanya, `mean` lebih cocok untuk distribusi normal (yang terlihat simetris di tengah), sementara median lebih baik untuk distribusi bersifat *skewed* (berat sebelah). Tentunya, ini tidak berlaku untuk data kategorik yang hanya bisa menggunakan `mode`.

Terkadang central tendency saja tidak cukup untuk melihat distribusi data. Central tendency tidak bisa menggambarkan seberapa besar data tersebar. **Measure of dispersion merupakan ukuran yang biasa digunakan untuk melihat keragaman data atau seberapa besar data tersebar.** Terdapat beberapa ukuran untuk menentukan dispersi data pengamatan, seperti jangkauan/rentang (`range`) dan simpangan baku (`standard deviation`).

Secara umum, berikut adalah contoh penggunaan descriptive statistics (1), (2), (3) dari atribut `AveragePrice`:

In [ ]:
# Array of 'AveragePrice' values
price_arr = avocado_df['AveragePrice']

price_stats = [
    ['Mean', np.mean(price_arr)],
    ['Median', np.median(price_arr)],
    ['Mode', scp.mode(price_arr)[0][0]],
    ['Standard Deviation', np.std(price_arr)],
    ['Variance', np.var(price_arr)],
    ['Minimum', np.min(price_arr)],
    ['Maximum', np.max(price_arr)],
]

pd.DataFrame(price_stats, columns=['Measure', 'Value'])

Pada hasil di atas, kita dapat lihat bahwa `mean` > `median` > `mode`. Data juga tersebar sebesar 0.4 dari nilai `mean`. Dengan nilai maksimum 3.25 dan minimum 0.44 ada kemungkinan bahwa data cenderung padat di nilai yang rendah atau padat ke kiri.

Diluar central tendency dan dispersion, statistik deskriptif juga mencakup ***skewness* yang mengukur kemiringan atau asimetri distribusi**. *Skewness* dari data dapat mempengaruhi pemrosesan, contohnya menangani *missing values* dengan *central tendency* yang tepat atau melakukan *sampling*. 

Cara melihat *skewness* dapat dilakukan melalui visualisasi KDE (Kernel Density Estimation) plot. KDE plot menunjukkan estimasi dari fungsi probabilitas *density* pada sebuah data yang bersifat kontinu.

In [ ]:
# KDE Plot (Distribusi data) dari AveragePrice

plt.figure(figsize=(7,5))
sns.kdeplot(price_arr, shade=True)
plt.vlines(x=np.mean(price_arr), ymin=0, ymax=1, color='blue', linestyle='--')
plt.vlines(x=np.median(price_arr), ymin=0, ymax=1, color='brown', linestyle='--')
plt.vlines(x=scp.mode(price_arr)[0][0], ymin=0, ymax=1, color='red', linestyle='--')
plt.legend(['AveragePrice', 'Mean', 'Median', 'Mode'], fontsize=12)
plt.show()

Terlihat bahwa distribusi bersifat skewed karena sedikit lebih berat di kiri. Berdasarkan visualisasi, median, mean, dan modus tidak selalu sama. Pada kasus ini, terlihat bahwa median dapat menjadi pengukuran yang paling tepat untuk menggambarkan central tendency dari data.

Selain itu, kita dapat mengecek nilai skewness dari atribut-atribut numerik dengan menggunakan fungsi `.skew()` berikut:

In [ ]:
# Mengambil hanya atribut float numerik
numeric_avocado = avocado_df.select_dtypes(include=['float64'])

# Mengecek nilai skewness dari data
numeric_avocado.skew()

Interpretasi nilai *skewness*:
*   Jika nilainya positif maka right skewed (buntut kanan > buntut kiri)
*   Jika nilainya negatif maka left skewed (buntut kiri > buntut kanan)
*   Jika skewness = 0, data simetris

Dari hasil tersebut, didapati bahwa AveragePrice memiliki karakteristik right skewed dengan tingkat asimetris yang tidak terlalu tinggi. Sementara atribut numerik lain memiliki asimetris yang sangat tinggi ke kanan.


Untuk membandingkan distribusi data A ke data B, kita dapat menggunakan overlapping Histogram atau KDE plot.

Misalkan kita ingin melihat perbandingan `AveragePrice` dari conventional vs organic avocados.

In [ ]:
conventional = avocado_df[avocado_df['type'] == "conventional"]
organic = avocado_df[avocado_df['type'] == "organic"]
conv_price = conventional['AveragePrice']
org_price = organic['AveragePrice']

# Histogram dengan "density=True" untuk menyerupai KDE Plot
plt.figure(figsize=(7,5))
plt.title("Average Prices per Avocado Type", fontsize=12)
plt.hist(conv_price, alpha=0.5, bins=50, density=True, label="Conventional")
plt.hist(org_price, alpha=0.5, bins=50, density=True, label="Organic")
plt.xlabel("AveragePrice")
plt.ylabel("Density")
plt.legend()
plt.show()

In [ ]:
# KDE Plot
plt.figure(figsize=(7,5))
sns.kdeplot(conv_price, shade=True)
sns.kdeplot(org_price, shade=True)
plt.legend(['Conventional','Organic'], fontsize=12)
plt.vlines(x=conv_price.mean(), ymin=0, ymax=1.6, color='blue', linestyle='--')
plt.vlines(x=org_price.mean(), ymin=0, ymax=1.2, color='brown', linestyle='--')
plt.show()

## Correlation

Korelasi merupakan pengukuran dependensi antar atribut dalam data. Korelasi sering digunakan untuk mengukur apakah suatu atribut memiliki hubungan yang kuat secara linier atau tidak. 

Terdapat beberapa jenis korelasi yang dapat digunakan pada jenis data yang berbeda. ***Pearson correlation*** umum digunakan untuk mengukur korelasi pada atribut numerik.

In [ ]:
# Menampilkan nilai korelasi atribut numerik (dari koefisien Pearson)
avocado_df.corr(method='pearson')

Kita sudah dapat melihat nilai korelasi pada matrix di atas. Akan tetapi, visualisasi dari matrix tersebut kurang enak untuk dilihat. Untuk itu, kita akan menggunakan visualisasi heatmap.

In [ ]:
# Visualisasi matriks korelasi dengan heatmap
plt.figure(figsize=(10,10))

sns.heatmap(avocado_df.select_dtypes(np.number).corr(), 
        annot=True,
        cbar=False,
        fmt="0.2f",
        cmap="YlGnBu",
        xticklabels=avocado_df.select_dtypes(np.number).columns,
        yticklabels=avocado_df.select_dtypes(np.number).columns)
plt.title("Correlation matrix")

Dari heatmap diatas, kita bisa menentukan pasangan atribut apa saja yang memiliki relasi yang tinggi (warna biru gelap). Untuk membuktikan, dibawah terdapat 3 scatterplot yang menunjukkan korelasi dengan `Total Volume`, yakni
* `Total Bags`: 0.96 (terlihat berbanding lurus naik)
* `AveragePrice`: -0.19 (ada sedikit tren terbalik tetapi semua hampir berkumpul di total volume 0 - 1)
* `year`: 0.01 (membingungkan)

In [ ]:
avocado_df.plot.scatter(x="Total Volume", y="Total Bags", figsize=(6,4))

Dari visualisasi tersebut kita dapat melihat bahwa `Total Bags` berbanding lurus dengan `Total Volume`. Semakin tinggi `Total Bags`, maka `Total Volume` juga akan meningkat (tentu sangat masuk akal).

In [ ]:
avocado_df.plot.scatter(x="Total Volume", y="AveragePrice", figsize=(6,4))

Antara `Total Volume` dan `AveragePrice` kita dapati bahwa terjadi hubungan yang negatif di mana peningkatan volume menyebabkan harga menurun. Hal ini mungkin terjadi karena penjualan yang agresif (mungkin seperti penjualan grosir dan sebagainya).

In [ ]:
avocado_df.plot.scatter(x="Total Volume", y="year", figsize=(6,4))

Dari visualisasi di atas, kita mungkin menemui perhitungan dan visualisasi yang agak membingungkan. Hal ini wajar terjadi dikarenakan tahun yang dianggap sebagai data numerik sedangkan lebih tepat dianggap sebagai data kategorik. Tipe data akan sangat mempengaruhi pemrosesan di EDA, maka kita perlu berhati-hati untuk menentukan tipe data yang sesuai.

## Data Visualization

In [ ]:
# Fungsi yang membuat plot untuk melihat distribusi frekuensi untuk satu kolom/atribut
def countplot(column, df):
    return sns.countplot(x=column, data=df)

In [ ]:
countplot('type', avocado_df)

Sekilas dari countplot, atribut `type` memiliki data yang sangat seimbang (*perfectly balanced as all things should be*).

In [ ]:
countplot('year', avocado_df)

Sedangkan, atribut `year` menandakan kalau data dari 2018 jauh lebih sedikit dibanding tahun-tahun sebelumnya. Hal ini wajar terjadi karena dataset memang diambil pada pertengahan tahun 2018.

In [ ]:
# Menampilkan boxplot type terhadap average price per tahun
sns.boxplot(x="type", y="AveragePrice", hue="year", data=avocado_df)

Terlihat bahwa avocado bertipe `organic` memiliki rata-rata harga (`AveragePrice`) yang lebih tinggi dibanding dengan yang bertipe `conventional` setiap tahunnya.

In [ ]:
# Menampilkan boxplot type terhadap total volume per tahun
sns.boxplot(x="type", y="Total Volume", hue="year", data=avocado_df)

Jika dilihat dari total volume yang terjual, terlihat sepertinya avocado memiliki "outliers" yang cukup banyak dan jauh dari IQR sampai boxplotnya sendiri tidak terlihat.

In [ ]:
# Menampilkan boxplot untuk atribut yang memiliki banyak unique values (seperti region) terhadap average price
fig = px.box(avocado_df, x='region', y='AveragePrice')
fig.show()

## Latihan Soal

### **SOAL 1: Descriptive Statistics**

(a) Tampilkan descriptive statistics dari atribut `Total Volume`! (hint: tambahkan range, Q1, Q2, Q3, 20th Percentile)

In [ ]:
# 1(a) Implementasi kode di sini

(b) Apa central tendency paling cocok untuk atribut `Total Volume`? Berikan alasan!

In [ ]:
# 1(b) Implementasi kode di sini

### **SOAL 2: Data Visualization**

Untuk setiap sub-soal, pilih salah satu dari visualisasi berikut:
* Pie chart
* Bar chart
* Scatter plot
* Box plot
* Histogram
* Line graph

Beberapa kasus terkadang memiliki lebih dari satu "jawaban yang tepat", karena ada banyak cara untuk visualisasi data. Pilihlah yang menurut Anda paling cocok berdasarkan kasus yang diberikan.

**Secara eksplisit, tulis chart apa yang Anda pilih dan alasannya.**

(a) Buatlah visualisasi untuk melihat rata-rata alpukat yang terjual di setiap region. Apakah California adalah region dengan rata-rata penjualan alpukat terbanyak? (Asumsi `TotalUS` bukanlah sebuah region)

(rev1: 14/09/2021 12:47 WIB)

In [ ]:
# 2(a) Implementasi kode di sini

(b) Buatlah visualisasi yang menunjukkan average price per tahun. Dari visualisasi, di antara 2 tahun apakah yang memiliki perbedaan average price yang tertinggi?

In [ ]:
# 2(b) Implementasi kode di sini

(c) Buatlah visualisasi dari persentase total volume alpukat yang terjual dengan average price >= 1.4 dari masing-masing tipe. Berapakah persentase dari organic avocadoes yang terjual dengan average price >= 1.4?

In [ ]:
# 2(c) Implementasi kode di sini

(d) Buatlah visualisasi untuk melihat pola dari `AveragePrice` dan `Small Bags`. Berdasarkan apa yang Anda lihat, apakah ada outlier(s) untuk alpukat dengan average price sekitar 1.0?

In [ ]:
# 2(d) Implementasi kode di sini

(e) Buatlah visualisasi untuk melihat apakah ada outliers di atribut `Total Bags` untuk setiap region. Berikan satu contoh region yang terlihat tidak memiliki outlier.

In [ ]:
# 2(e) Implementasi kode di sini

(f) Lihatlah perbandingan distribusi data dari jumlah alpukat yang dijual di California dengan GreatLakes. Berikan analisis Anda.

In [ ]:
# 2(f) Implementasi kode di sini